In [1]:
import pandas as pd
import datetime
import numpy as np
import datetime
epoch = datetime.datetime.utcfromtimestamp(0)

In [3]:
# !pip install fastai==0.7.0

In [4]:
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from IPython.display import display

## Training Data Prep

In [15]:
df = pd.read_csv('/Users/nialdaly/Documents/ey_nextwave/data/data_train.csv',
                 parse_dates=['time_exit', 'time_entry'])

df.drop(['Unnamed: 0'],axis=1,inplace=True)

df.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,2019-04-25 07:04:31,2019-04-25 07:08:32,NaN,NaN,NaN,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,2019-04-25 07:20:34,2019-04-25 07:25:42,NaN,NaN,NaN,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,2019-04-25 07:53:32,2019-04-25 08:03:25,NaN,NaN,NaN,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,2019-04-25 08:17:50,2019-04-25 08:37:23,NaN,NaN,NaN,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,2019-04-25 14:38:09,2019-04-25 14:38:09,NaN,NaN,NaN,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07


## Reusing test data as training data

In [ ]:
useful_test_df = pd.read_csv('/Users/nialdaly/Documents/ey_nextwave/data/data_test.csv',
                 parse_dates=['time_exit', 'time_entry'])

useful_test_df.head()

In [11]:
len(useful_test_df)

169422

In [13]:
# removes the Unnamed column 
useful_test_df.drop(['Unnamed: 0'],axis=1,inplace=True)

# removes all rows with no data in the following columns
useful_test_df = useful_test_df.dropna(subset=['x_exit', 'y_exit'])

useful_test_df.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0,2019-04-25 11:43:17,2019-04-25 11:50:17,NaN,NaN,NaN,3.773413e+06,-1.909828e+07,3.773111e+06,-1.914508e+07
1,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_2,2019-04-25 12:21:37,2019-04-25 12:21:37,0.00,0.00,0.00,3.773199e+06,-1.914354e+07,3.773199e+06,-1.914354e+07
2,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_3,2019-04-25 12:34:27,2019-04-25 13:14:11,NaN,NaN,NaN,3.763760e+06,-1.921342e+07,3.771757e+06,-1.911092e+07
3,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4,2019-04-25 13:25:33,2019-04-25 13:43:13,NaN,NaN,NaN,3.773385e+06,-1.911344e+07,3.773131e+06,-1.914465e+07
5,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_0,2019-04-25 08:08:23,2019-04-25 08:20:08,17.29,17.29,17.29,3.771380e+06,-1.933274e+07,3.769933e+06,-1.934095e+07


In [14]:
len(useful_test_df)

169422

## Data concatenation

Concatenates the df and useful_test_df based on rows

In [16]:
df = pd.concat([df, useful_test_df])

# reindexes the dataframe
df = df.reset_index(drop=True)

df.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,2019-04-25 07:04:31,2019-04-25 07:08:32,NaN,NaN,NaN,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,2019-04-25 07:20:34,2019-04-25 07:25:42,NaN,NaN,NaN,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,2019-04-25 07:53:32,2019-04-25 08:03:25,NaN,NaN,NaN,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,2019-04-25 08:17:50,2019-04-25 08:37:23,NaN,NaN,NaN,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,2019-04-25 14:38:09,2019-04-25 14:38:09,NaN,NaN,NaN,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07


In [18]:
len(df)

983684

## Data preprocessing

In [19]:
# 
def unix_time_millis(dt):
    return (dt - epoch).total_seconds() * 1000.0

In [20]:
def multiply(row):
    return row['x_entry'] * row['y_entry']

In [23]:
#
# multxy = np.zeros(814262)
multxy = np.zeros(983684)

#
df['multxy'] = multxy

df.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,multxy
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,2019-04-25 07:04:31,2019-04-25 07:08:32,NaN,NaN,NaN,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07,0.0
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,2019-04-25 07:20:34,2019-04-25 07:25:42,NaN,NaN,NaN,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07,0.0
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,2019-04-25 07:53:32,2019-04-25 08:03:25,NaN,NaN,NaN,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07,0.0
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,2019-04-25 08:17:50,2019-04-25 08:37:23,NaN,NaN,NaN,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07,0.0
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,2019-04-25 14:38:09,2019-04-25 14:38:09,NaN,NaN,NaN,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07,0.0


In [24]:
#
df['multxy'] = df.apply(multiply, axis=1)

df.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,multxy
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,2019-04-25 07:04:31,2019-04-25 07:08:32,NaN,NaN,NaN,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07,-7.162178e+13
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,2019-04-25 07:20:34,2019-04-25 07:25:42,NaN,NaN,NaN,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07,-7.234210e+13
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,2019-04-25 07:53:32,2019-04-25 08:03:25,NaN,NaN,NaN,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07,-7.225185e+13
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,2019-04-25 08:17:50,2019-04-25 08:37:23,NaN,NaN,NaN,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07,-7.224731e+13
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,2019-04-25 14:38:09,2019-04-25 14:38:09,NaN,NaN,NaN,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07,-7.222276e+13


In [25]:
# encoding time
df_new_entry = df['time_entry'].apply(unix_time_millis)
df_new_exit = df['time_exit'].apply(unix_time_millis)

# applies the new time encodings to each column
df['time_entry'] = df_new_entry
df['time_exit'] = df_new_exit

df.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,multxy
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,1.556176e+12,1.556176e+12,NaN,NaN,NaN,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07,-7.162178e+13
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,1.556177e+12,1.556177e+12,NaN,NaN,NaN,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07,-7.234210e+13
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,1.556179e+12,1.556179e+12,NaN,NaN,NaN,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07,-7.225185e+13
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,1.556180e+12,1.556181e+12,NaN,NaN,NaN,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07,-7.224731e+13
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,1.556203e+12,1.556203e+12,NaN,NaN,NaN,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07,-7.222276e+13


In [26]:
df['y'] = np.where(
    (df['x_exit'] > 3750901.5068) & (df['x_exit'] < 3770901.5068) &
    (df['y_exit'] >= -19268905.6133) & (df['y_exit'] <= -19208905.6133), 
    1, 0)

In [29]:
#
df_clean = df.dropna(subset=['time_entry','time_exit','x_entry','y_entry'])

#
# df_clean.drop(['Unnamed: 0'],axis=1,inplace=True)

df_clean.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,multxy,y
0,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_0,1.556176e+12,1.556176e+12,NaN,NaN,NaN,3.751014e+06,-1.909398e+07,3.750326e+06,-1.913634e+07,-7.162178e+13,0
1,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_1,1.556177e+12,1.556177e+12,NaN,NaN,NaN,3.743937e+06,-1.932247e+07,3.744975e+06,-1.931966e+07,-7.234210e+13,0
2,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_2,1.556179e+12,1.556179e+12,NaN,NaN,NaN,3.744868e+06,-1.929356e+07,3.744816e+06,-1.929284e+07,-7.225185e+13,0
3,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_3,1.556180e+12,1.556181e+12,NaN,NaN,NaN,3.744880e+06,-1.929229e+07,3.744809e+06,-1.929049e+07,-7.224731e+13,0
4,0000a8602cf2def930488dee7cdad104_1,traj_0000a8602cf2def930488dee7cdad104_1_4,1.556203e+12,1.556203e+12,NaN,NaN,NaN,3.744909e+06,-1.928558e+07,3.744909e+06,-1.928558e+07,-7.222276e+13,0


In [30]:
#
df_clean.to_csv('/Users/nialdaly/Documents/ey_nextwave/data/data_train_cleaned.csv')

## Test Data Prep

In [2]:
test_df = pd.read_csv('/Users/nialdaly/Documents/ey_nextwave/data/data_test.csv',
                 parse_dates=['time_exit', 'time_entry'])

# removes the Unnamed column 
test_df.drop(['Unnamed: 0'],axis=1,inplace=True)

test_df.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0,2019-04-25 11:43:17,2019-04-25 11:50:17,NaN,NaN,NaN,3.773413e+06,-1.909828e+07,3.773111e+06,-1.914508e+07
1,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_2,2019-04-25 12:21:37,2019-04-25 12:21:37,0.0,0.0,0.0,3.773199e+06,-1.914354e+07,3.773199e+06,-1.914354e+07
2,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_3,2019-04-25 12:34:27,2019-04-25 13:14:11,NaN,NaN,NaN,3.763760e+06,-1.921342e+07,3.771757e+06,-1.911092e+07
3,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4,2019-04-25 13:25:33,2019-04-25 13:43:13,NaN,NaN,NaN,3.773385e+06,-1.911344e+07,3.773131e+06,-1.914465e+07
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,2019-04-25 15:03:32,2019-04-25 15:10:32,NaN,NaN,NaN,3.773118e+06,-1.914490e+07,NaN,NaN


In [22]:
# removes duplicates in the test data based on the hash column
test_df = test_df.drop_duplicates(subset='hash', keep='last')

test_df.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,2019-04-22 15:03:32,2019-04-22 15:10:32,NaN,NaN,NaN,3.773118e+06,-1.914490e+07,NaN,NaN
7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,2019-04-22 15:29:09,2019-04-22 15:30:56,NaN,NaN,NaN,3.769978e+06,-1.934136e+07,NaN,NaN
10,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,2019-04-22 15:26:08,2019-04-22 15:26:08,NaN,NaN,NaN,3.757468e+06,-1.923860e+07,NaN,NaN
13,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,2019-04-22 15:35:18,2019-04-22 15:42:05,NaN,NaN,NaN,3.760505e+06,-1.935500e+07,NaN,NaN
17,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,2019-04-22 14:54:07,2019-04-22 15:05:14,NaN,NaN,NaN,3.766319e+06,-1.917013e+07,NaN,NaN


In [23]:
#
multxy = np.zeros(len(test_df))

#
test_df['multxy'] = multxy

test_df.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,multxy
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,2019-04-22 15:03:32,2019-04-22 15:10:32,NaN,NaN,NaN,3.773118e+06,-1.914490e+07,NaN,NaN,0.0
7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,2019-04-22 15:29:09,2019-04-22 15:30:56,NaN,NaN,NaN,3.769978e+06,-1.934136e+07,NaN,NaN,0.0
10,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,2019-04-22 15:26:08,2019-04-22 15:26:08,NaN,NaN,NaN,3.757468e+06,-1.923860e+07,NaN,NaN,0.0
13,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,2019-04-22 15:35:18,2019-04-22 15:42:05,NaN,NaN,NaN,3.760505e+06,-1.935500e+07,NaN,NaN,0.0
17,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,2019-04-22 14:54:07,2019-04-22 15:05:14,NaN,NaN,NaN,3.766319e+06,-1.917013e+07,NaN,NaN,0.0


In [24]:
test_df['multxy'] = test_df.apply(multiply, axis=1)

test_df.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,multxy
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,2019-04-22 15:03:32,2019-04-22 15:10:32,NaN,NaN,NaN,3.773118e+06,-1.914490e+07,NaN,NaN,-7.223597e+13
7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,2019-04-22 15:29:09,2019-04-22 15:30:56,NaN,NaN,NaN,3.769978e+06,-1.934136e+07,NaN,NaN,-7.291652e+13
10,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,2019-04-22 15:26:08,2019-04-22 15:26:08,NaN,NaN,NaN,3.757468e+06,-1.923860e+07,NaN,NaN,-7.228841e+13
13,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,2019-04-22 15:35:18,2019-04-22 15:42:05,NaN,NaN,NaN,3.760505e+06,-1.935500e+07,NaN,NaN,-7.278458e+13
17,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,2019-04-22 14:54:07,2019-04-22 15:05:14,NaN,NaN,NaN,3.766319e+06,-1.917013e+07,NaN,NaN,-7.220082e+13


In [25]:
# encoding time
df_new_entry = test_df['time_entry'].apply(unix_time_millis)
df_new_exit = test_df['time_exit'].apply(unix_time_millis)

# applies the new time encodings to each column
test_df['time_entry'] = df_new_entry
test_df['time_exit'] = df_new_exit

test_df.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,multxy
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,1.555945e+12,1.555946e+12,NaN,NaN,NaN,3.773118e+06,-1.914490e+07,NaN,NaN,-7.223597e+13
7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,1.555947e+12,1.555947e+12,NaN,NaN,NaN,3.769978e+06,-1.934136e+07,NaN,NaN,-7.291652e+13
10,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,1.555947e+12,1.555947e+12,NaN,NaN,NaN,3.757468e+06,-1.923860e+07,NaN,NaN,-7.228841e+13
13,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,1.555947e+12,1.555948e+12,NaN,NaN,NaN,3.760505e+06,-1.935500e+07,NaN,NaN,-7.278458e+13
17,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,1.555945e+12,1.555946e+12,NaN,NaN,NaN,3.766319e+06,-1.917013e+07,NaN,NaN,-7.220082e+13


In [26]:
#
test_df = test_df.dropna(subset=['time_entry','time_exit','x_entry','y_entry'])

test_df.head()

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit,multxy
4,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_5,1.555945e+12,1.555946e+12,NaN,NaN,NaN,3.773118e+06,-1.914490e+07,NaN,NaN,-7.223597e+13
7,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_10,1.555947e+12,1.555947e+12,NaN,NaN,NaN,3.769978e+06,-1.934136e+07,NaN,NaN,-7.291652e+13
10,000506a39775e5bca661ac80e3f466eb_29,traj_000506a39775e5bca661ac80e3f466eb_29_5,1.555947e+12,1.555947e+12,NaN,NaN,NaN,3.757468e+06,-1.923860e+07,NaN,NaN,-7.228841e+13
13,0005401ceddaf27a9b7f0d42ef1fbe95_1,traj_0005401ceddaf27a9b7f0d42ef1fbe95_1_4,1.555947e+12,1.555948e+12,NaN,NaN,NaN,3.760505e+06,-1.935500e+07,NaN,NaN,-7.278458e+13
17,00063a4f6c12e1e4de7d876580620667_3,traj_00063a4f6c12e1e4de7d876580620667_3_4,1.555945e+12,1.555946e+12,NaN,NaN,NaN,3.766319e+06,-1.917013e+07,NaN,NaN,-7.220082e+13


In [27]:
#
test_df.to_csv('/Users/nialdaly/Documents/ey_nextwave/data/data_test_cleaned.csv')

202937

,hash,trajectory_id,time_entry,time_exit,vmax,vmin,vmean,x_entry,y_entry,x_exit,y_exit
0,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_0,2019-04-25 11:43:17,2019-04-25 11:50:17,NaN,NaN,NaN,3.773413e+06,-1.909828e+07,3.773111e+06,-1.914508e+07
1,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_2,2019-04-25 12:21:37,2019-04-25 12:21:37,0.00,0.00,0.00,3.773199e+06,-1.914354e+07,3.773199e+06,-1.914354e+07
2,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_3,2019-04-25 12:34:27,2019-04-25 13:14:11,NaN,NaN,NaN,3.763760e+06,-1.921342e+07,3.771757e+06,-1.911092e+07
3,00032f51796fd5437b238e3a9823d13d_31,traj_00032f51796fd5437b238e3a9823d13d_31_4,2019-04-25 13:25:33,2019-04-25 13:43:13,NaN,NaN,NaN,3.773385e+06,-1.911344e+07,3.773131e+06,-1.914465e+07
5,000479418b5561ab694a2870cc04fd43_25,traj_000479418b5561ab694a2870cc04fd43_25_0,2019-04-25 08:08:23,2019-04-25 08:20:08,17.29,17.29,17.29,3.771380e+06,-1.933274e+07,3.769933e+06,-1.934095e+07


In [10]:
len(useful_test_df)

169422